detecting fake news using logistic regression (since it is a binary classification problem)

import the dependencies

In [126]:
import numpy as np # for numpy array (n-D array)
import pandas as pd # for structuring the data into tabular format
import re # for identifying words from a paragraph

from nltk.corpus import stopwords # those words which add a very little value to the given dataset
from nltk.stem.porter import PorterStemmer # removing prefix and postfix to get the root word

from sklearn.feature_extraction.text import TfidfVectorizer # converting the root to numerical form
from sklearn.model_selection import train_test_split # spliting the data into train and test dataset
from sklearn.linear_model import LogisticRegression # using classification model (logistic regression in this case)
from sklearn.metrics import accuracy_score # to check for the accuracy of the model upon the training and testing data

In [127]:
import nltk # nltk means natural language toolkit
nltk.download("stopwords") # downloading the set of all the stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [128]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

data collection and preprocessing

In [129]:
from google.colab import drive
drive.mount("/content/data")

Drive already mounted at /content/data; to attempt to forcibly remount, call drive.mount("/content/data", force_remount=True).


In [130]:
fake_news_dataset = pd.read_csv("/content/data/MyDrive/self_projects/fake_news/train.csv")
fake_news_dataset.head()

id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1

1: Fake news<br>
0: Real news

In [131]:
fake_news_dataset.shape

(20800, 5)

In [132]:
fake_news_dataset.isnull().sum() # checking for any null items

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [133]:
fake_news_dataset.fillna("", inplace=True) # replacing the null values with empty string

In [134]:
fake_news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

spliting the dataset into input features and labelled data

In [135]:
X = fake_news_dataset.drop(columns="label", axis=1)
y = fake_news_dataset["label"]

In [136]:
X["content"] = X["author"] +" "+ X["title"] # instead of text as the input data coz its huge, we take author and title instead

In [137]:
X["content"]

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [138]:
X.shape

(20800, 5)

stemming: the process of reducing a word to its root word

In [139]:
porter_stem = PorterStemmer()
stop_words = stopwords.words("english")

In [140]:
def stemming(content):
  data_content = re.sub("[^a-zA-Z]", " ", content) # substituting non alpha (numerical and special characters) data with 1 blank space
  data_content = data_content.lower() # converting the whole text to lowercase
  data_content = data_content.split() # splitting the data to retrieve them in the form of a list
  data_content = [porter_stem.stem(words) for words in data_content if not words in stop_words] # actual stemming process plus removing the stopwords as well
  # in the form of a python list
  data_content = " ".join(data_content)
  return data_content

In [141]:
X["content"] = X["content"].apply(stemming)

In [142]:
print(X["content"])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [143]:
X = X["content"].values

In [144]:
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [145]:
vectorize = TfidfVectorizer() # Term Frequency Inverse Document Frequency Records
# helps to convert textual data to numerical data
vectorize.fit(X)
X = vectorize.transform(X)

In [146]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [102]:
print(y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


In [147]:
X.shape

(20800, 17128)

In [148]:
y.shape

(20800,)

In [149]:
y = y.values

In [150]:
y

array([1, 0, 1, ..., 0, 1, 1])

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

In [152]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16640, 17128), (4160, 17128), (16640,), (4160,))

selecting a model: Logistic Regression

In [153]:
lor = LogisticRegression()
lor.fit(X_train, y_train)

LogisticRegression()

model evaluation

In [154]:
# training data
X_train_prediction = lor.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
print("Accuracy score for training data: ", training_data_accuracy)

Accuracy score for training data:  0.9865985576923076


In [155]:
# testing data
X_test_prediction = lor.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, y_test)
print("Accuracy score for testing data: ", testing_data_accuracy)

Accuracy score for testing data:  0.9790865384615385


make a predictive system

In [156]:
x_new = X_test[0]

prediction = lor.predict(x_new)
print(prediction)

if prediction[0] == 0:
  print("The news is real")
else:
  print("The news is fake")

[1]
The news is fake


In [157]:
print(y_test[0])

1


Thus we can that our model is predicting correctly